# Noise
> Load and scale 1/f and white noise simulations
- toc: True

* They include white and 1/f noise
* Simulated for 10 days, see note about efficiency for atmosphere
* also have a thinning factor on the number of channels

In [ ]:
# default_exp noise

In [ ]:
import logging as log

In [ ]:
# export

import healpy as hp
import numpy as np
from pathlib import Path
import logging as log

from s4refsimtool.core import get_telescope, base_folder, simulations_observing_efficiency

In [ ]:
# export

thinfp_string = """telescope  band   thinfp
LAT        HFL      8
LAT        MFL      8
LAT        LFL      1
LAT        ULFL     1
SAT        HFS      8
SAT        MFHS     4
SAT        MFLS     4
SAT        LFS      1
"""

from astropy.io import ascii

thinfp_table = ascii.read(thinfp_string)

In [ ]:
# exports


def get_thinfp(channel):
    """Get the focal plane thinning factor for noise simulations

    Parameters
    ----------
    channel : str
        CMB-S4 channel tag e.g. HFL2

    Returns
    -------
    thinfp : int
        thinning factor
    """
    return (thinfp_table[thinfp_table["band"] == channel[:-1]])["thinfp"]

In [ ]:
assert get_thinfp("ULFL1") == 1
assert get_thinfp("HFL2") == 8
assert get_thinfp("MFLS1") == 4

In [ ]:
# exports


def get_tube_years(config, site, channel):
    """Compute the number of tube/years in the CMB-S4 configuration

    config_telescopes : dict
        CMB-S4 telescopes configuration,
        generally loaded from a TOML file
    site : str
        'Pole' or 'Chile', case doesn't matter
    channel : str
        Channel tag, e.g. 'MFHS1'
    """
    tube_years = 0
    for tube, bands in config["telescopes"][get_telescope(channel)].items():
        if bands["site"].lower() == site.lower():
            num_tubes = bands.get(channel[:-1], 0)
            tube_years += num_tubes * bands.get(
                "years", config["experiment"]["total_experiment_length_years"]
            )
    return tube_years

In [ ]:
import h5py

s4 = h5py.File("cmbs4_tophat.h5", 'r')

In [ ]:
!tail -30 s4_reference_design.toml

In [ ]:
import toml

config = toml.load("s4_reference_design.toml")

In [ ]:
log.basicConfig(level=log.INFO)

In [ ]:
for site in ["Pole", "Chile"]:
    for channel in s4.keys():
        tube_years = get_tube_years(config, site, channel)
        print(site, channel, tube_years)
        telescope = get_telescope(channel)
        if site == "Chile":
            if telescope == "SAT":
                assert tube_years == 0, "All SAT at Pole"
            elif channel.startswith("ULFL"):
                assert tube_years == 0, "No ULFL in Chile"              
            else:
                if channel.startswith("MFL"):
                    assert tube_years == 24*7, "12+12 MFL channels"
        if site == "Pole":
            if telescope == "SAT":
                if channel.startswith("LFS"):
                    assert tube_years == 2*7, "2 LFS Tubes"
                elif channel.startswith("MF"):
                    assert tube_years == 6*7, "2 LFS Tubes"                  
            else:
                if channel.startswith("HFL"):
                    assert tube_years == 4*7, "4 HFL tubes"            

In [ ]:
# exports


def load_noise(config, site, channel, realization=0):
    """Load noise maps for a channel

    Parameters
    ----------
    config : dict
        CMB-S4 configuration,
        generally loaded from a TOML file
    site : str
        'Pole' or 'Chile', case doesn't matter
    channel : str
        Channel tag, e.g. 'MFHS1'
    realization : int
        Choose one of the available 8 realizations

    Returns
    -------
    output_map : numpy array
        Output map with all emissions combined, uses nan for missing pixels
    """
    telescope = get_telescope(channel)
    map_filename = (
        Path(f"{realization:08d}")
        / f"{site.lower()}_noise_{telescope}_{channel}_filtered_telescope_all_time_all_bmap.fits"
    )
    log.info(f"Reading {map_filename}")
    output_map = hp.read_map(
        Path(base_folder) / map_filename, (0, 1, 2), dtype=np.float32, verbose=False
    )
    output_map[output_map == hp.UNSEEN] = np.nan
    # input map is 10 days at 100% efficiency
    output_map *= 10 / (365.25 * config["experiment"]["observing_efficiency"])
    output_map *= get_tube_years(config, site, channel)
    # pole sims have lower efficiency
    output_map *= np.sqrt(
        simulations_observing_efficiency[site.lower()].get(telescope, 1)
    )
    # focal plane thinning factor of TOD simulations
    output_map /= np.sqrt(get_thinfp(channel))
    return output_map

## Available input maps

In [ ]:
filenames = !ls /global/cscratch1/sd/keskital/s4sim/reference_tool/out/00000000/*noise*bmap*

In [ ]:
import os.path
for f in map(os.path.basename, filenames):
    print(f)

In [ ]:
channel = "LFS1"
site = "Pole"

In [ ]:
output_map = load_noise(config, site, channel, realization=0)

In [ ]:
np.nanmin(output_map), np.nanmax(output_map)

In [ ]:
assert np.nanmax(output_map) < 1e-2 and np.nanmax(output_map) > 1e-3, \
    "Amplitude check failed"

In [ ]:
assert np.nanmin(output_map) > -1e-2 and np.nanmin(output_map) < 1e-3, \
    "Amplitude check failed"

In [ ]:
%matplotlib inline

In [ ]:
hp.mollview(output_map[0], min=-1e-4, max=1e-4, unit="K", title="Noise I")

In [ ]:
hp.mollview(output_map[1], min=-1e-4, max=1e-4, unit="K", title="Noise Q")

In [ ]:
hp.mollview(output_map[2], min=-1e-4, max=1e-4, unit="K", title="Noise U")